<a href="https://colab.research.google.com/github/41371122h-lichi/lichi_thursday/blob/main/HW6_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課表查詢與提醒 (作業六)

以下是此程式可以使用的功能：

1.   匯入PDF並將課表上傳至指定的google sheet
2.   輸入本週課程的作業/複習章節/備註
1.   輸入下週課程的應帶用品/預習章節/備註
2.   讓AI小助手提醒你要帶的東西
1.   讓AI小助手推薦給你跟課程有關的影片和文章吧

p.s 我們親愛的Gemini還不太穩定，若是有把「統計學(一)」分為「統計學」和「統計學(一)」的時候，再請各位多諒解了~








In [132]:
!pip install google-genai pytz gradio gspread pandas

In [133]:
import os, time, uuid, re, json
import pandas as pd
import pytz
from datetime import datetime as dt_class, timedelta, date
from dateutil.tz import gettz
import gradio as gr

import google.generativeai as genai
from google.colab import auth
import gspread
from google.auth import default
from google.colab import userdata
from google.colab import files

In [134]:
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google 服務授權完成")
except Exception as e:
    print(f"❌ Google 服務授權失敗，請確認 Colab 認證：{e}")
    gc = None

✅ Google 服務授權完成


In [135]:
try:
    api_key = userdata.get('gemini')
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-flash')
    print(f"✅ Gemini API 配置完成")
except Exception as e:
    print(f"❌ Gemini API 配置失敗。請檢查 Colab Secrets 中的 'gemini' 密鑰: {e}")
    model = None

✅ Gemini API 配置完成


In [136]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1zfGFuyvL1T1WoeoUTHWMtlZH01K5AzcyoNDef0-ylxE/edit?usp=sharing"
WORKSHEET_NAME_CURRENT = "本週課表"
WORKSHEET_NAME_NEXT = "下週課表"
TIMEZONE = "Asia/Taipei"
TW_TZ = pytz.timezone(TIMEZONE)

COLUMNS_CURRENT = ["日期","星期","課程名稱","時間(起)","時間(迄)","地點","作業","複習章節","備註"]
COLUMNS_NEXT    = ["日期","星期","課程名稱","時間(起)","時間(迄)","地點","應帶用品","預習章節","備註"]
COMMON_COLS = ["日期","星期","課程名稱","時間(起)","時間(迄)","地點","備註"]
ALL_COLUMNS = list(set(COLUMNS_CURRENT + COLUMNS_NEXT))

In [137]:
# 日期運算
today = dt_class.now(TW_TZ).date()
current_week_start_date = today - timedelta(days=today.weekday())
current_week_end_date = current_week_start_date + timedelta(days=6)
next_week_start_date = current_week_start_date + timedelta(days=7)
next_week_end_date = next_week_start_date + timedelta(days=6)

START_DATE_CURRENT = current_week_start_date
END_DATE_CURRENT = current_week_end_date
START_DATE_NEXT = next_week_start_date
END_DATE_NEXT = next_week_end_date

# Gradio Dropdown的日期選項
DATE_OPTIONS = {}
def get_date_options():
    date_options = {}
    for i in range(7):
        current_date = START_DATE_CURRENT + timedelta(days=i)
        date_str = current_date.strftime("%Y-%m-%d")
        label = f"本週 {current_date.strftime('%Y-%m-%d')} ({['一', '二', '三', '四', '五', '六', '日'][i]})"
        date_options[label] = date_str

    for i in range(7):
        next_date = START_DATE_NEXT + timedelta(days=i)
        date_str = next_date.strftime("%Y-%m-%d")
        label = f"下週 {next_date.strftime('%Y-%m-%d')} ({['一', '二', '三', '四', '五', '六', '日'][i]})"
        date_options[label] = date_str
    return date_options

DATE_OPTIONS = get_date_options()

In [138]:
# 統一處理日期和欄位
def standardize_df(df, expected_cols):
    if df.empty: return pd.DataFrame()

    TEXT_COLUMNS_TO_CLEAN = ["作業", "複習章節", "應帶用品", "預習章節", "備註"]

    for col in ALL_COLUMNS:
        if col not in df.columns:
            df[col] = ''
        elif col in TEXT_COLUMNS_TO_CLEAN:
             df[col] = df[col].astype(str).replace({'nan': ''}).fillna('')

    def ensure_datetime(s):
        if not s: return pd.NaT
        try:
            s = str(s).strip().replace("/", "-")

            date_obj = pd.to_datetime(s, errors='coerce')

            if pd.notna(date_obj):
                return date_obj.date()
            return pd.NaT
        except Exception:
            return pd.NaT

    df["日期"] = df["日期"].apply(ensure_datetime)
    df = df.dropna(subset=["日期"])
    return df[ALL_COLUMNS]

# 從 Sheets 載入數據
def load_sheet_to_df(worksheet_name):
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        sheets = gsheets.worksheet(worksheet_name).get_all_values()
        df = pd.DataFrame(sheets[1:], columns=sheets[0])
        return df
    except Exception as e:
        print(f"載入分頁 {worksheet_name} 失敗: {e}")
        return pd.DataFrame()

# 初始載入所有數據
def initial_data_load():
    df_current = standardize_df(load_sheet_to_df(WORKSHEET_NAME_CURRENT), COLUMNS_CURRENT)
    df_next = standardize_df(load_sheet_to_df(WORKSHEET_NAME_NEXT), COLUMNS_NEXT)
    df_all = pd.concat([df_current, df_next], ignore_index=True)
    return df_all, df_current, df_next

# 提取課表主函式
def process_week_with_gemini(start_date, end_date, worksheet_name, column_structure, pdf_path):
    if model is None or gc is None:
        return pd.DataFrame(columns=column_structure), f"❌ 環境錯誤：Gemini 或 Google Sheets 服務未配置/授權。"

    print(f"\n--- ⚙️ 處理 {worksheet_name} ({start_date} ~ {end_date}) ---")

    start_date_str = start_date.strftime("%Y/%m/%d")
    end_date_str = end_date.strftime("%Y/%m/%d")

    helper_cols = ",".join([c for c in column_structure if c not in COMMON_COLS])
    csv_header = ",".join(column_structure)

    prompt_text = f"""
    你是一位專業的資料擷取助理，任務是**忠實地**從我上傳的 PDF 課表檔案中提取資訊。

    **嚴格要求：你提取的內容，尤其是「課程名稱」和「地點」，必須完全存在於 PDF 文件中，不得進行任何推測、創造、美化或更改。**

    **請只**幫我擷取日期範圍在 **{start_date_str} 到 {end_date_str}** 之間的課表紀錄。

    你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，不要有任何其他說明文字。
    CSV 格式的欄位必須如下：
    "{csv_header}"

    - **關鍵指令：** 如果同一天有**完全相同課程名稱和地點**的課堂時段是**連續的**，請務必將它們**合併成一行紀錄**，使用**第一堂課的開始時間**和**最後一堂課的結束時間**。**請嚴格確保，所有課堂的課程名稱必須與 PDF 中的文字完全一樣，不得簡寫、省略括號或任何字元**
    - 「{helper_cols}」和「備註」這幾欄請保持空白，不要有任何標點符號。
    - 「日期」請使用 YYYY/MM/DD 格式。
    """

    gemini_file = None
    try:
        gemini_file = genai.upload_file(path=pdf_path, display_name="Gradio Uploaded Schedule PDF")
        response = model.generate_content([prompt_text, gemini_file])
        csv_output = response.text.strip()
        if csv_output.startswith("```csv"):
             csv_output = csv_output.replace("```csv\n", "").replace("```", "")

        temp_csv_file = f"gemini_schedule_{worksheet_name}.csv"
        with open(temp_csv_file, "w", encoding='utf-8') as f:
            f.write(csv_output)

        try:
            try:
                df_to_upload = pd.read_csv(temp_csv_file, engine='python', on_bad_lines='skip', sep=',', quotechar='"')
            except Exception:
                df_to_upload = pd.read_csv(temp_csv_file, engine='python', on_bad_lines='skip', sep=',', quotechar='"', skiprows=1, names=column_structure)

            df_to_upload = df_to_upload.dropna(how='all')
            df_to_upload = df_to_upload.fillna('')

            for col in ["日期", "課程名稱", "地點"]:
                if col not in df_to_upload.columns:
                    raise KeyError(f"❌ 嚴重錯誤：CSV 中缺少必要的欄位 '{col}'。")

            helper_cols_to_clear = [c for c in column_structure if c not in COMMON_COLS and c != "備註"]
            for col in helper_cols_to_clear:
                if col in df_to_upload.columns:
                    df_to_upload[col] = ''

            def simplify_course_name(name):
                name = re.sub(r'\(.*?\)', '', name)
                name = re.sub(r'（.*?）', '', name)
                return name.strip()

            df_to_upload['課程名稱_GROUP'] = df_to_upload['課程名稱'].apply(simplify_course_name)

            current_GROUP_KEYS = ["日期", "課程名稱", "地點"]
            columns_to_keep_first = [c for c in column_structure if c not in ["時間(起)", "時間(迄)"]]
            agg_rules = {col: 'first' for col in column_structure if col not in current_GROUP_KEYS}
            agg_rules['時間(起)'] = 'min'
            agg_rules['時間(迄)'] = 'max'

            df_merged = df_to_upload.groupby(current_GROUP_KEYS, as_index=False).agg(agg_rules)
            df_merged = df_merged.drop(columns=['課程名稱_GROUP'], errors='ignore')
            df_to_upload = df_merged[column_structure]
            print(f"✅ Pandas 已將時段強制合併，最終寫入資料共 {len(df_to_upload)} 筆。")

            df_to_upload = df_to_upload.sort_values(by=['日期', '時間(起)'], ascending=True)

            print(f"✅ 數據已按日期和時間排序完成。")

            gsheets = gc.open_by_url(SHEET_URL)
            try:
                worksheet = gsheets.worksheet(worksheet_name)
            except gspread.exceptions.WorksheetNotFound:
                worksheet = gsheets.add_worksheet(title=worksheet_name, rows=len(df_to_upload)+50, cols=len(df_to_upload.columns)+5)

            data_to_write = [df_to_upload.columns.values.tolist()] + df_to_upload.values.tolist()
            worksheet.clear()
            worksheet.update(data_to_write)

            success_msg = f"✅ 成功將課表內容寫入至 '{worksheet_name}' 分頁！"
            print(success_msg)

            return df_to_upload, success_msg

        except Exception as e:
            err_msg = f"❌ 處理 CSV 或寫入 Sheets 發生錯誤：{e}"
            print(err_msg)
            return pd.DataFrame(columns=column_structure), err_msg

    except Exception as e:
        err_msg = f"❌ Gemini 課表生成發生錯誤：{e}"
        print(err_msg)
        return pd.DataFrame(columns=column_structure), err_msg

    finally:
        if gemini_file:
            genai.delete_file(gemini_file.name)
            print("✅ Gemini 檔案已清理。")

In [139]:
def handle_pdf_upload(pdf_file_obj):
    if pdf_file_obj is None:
        return "⚠️ 請先上傳 PDF 檔案。", pd.DataFrame(), pd.DataFrame(), pd.DataFrame() # message, preview, df_all, df_current, df_next

    pdf_path = pdf_file_obj.name
    message = f"✅ 成功接收檔案: '{os.path.basename(pdf_path)}'。開始處理...\n"

    all_data_frames = []
    messages = []
    final_message = ""

    df_current_week, msg_current = process_week_with_gemini(
        START_DATE_CURRENT, END_DATE_CURRENT, WORKSHEET_NAME_CURRENT, COLUMNS_CURRENT, pdf_path
    )

    df_current_week = standardize_df(df_current_week, COLUMNS_CURRENT)
    all_data_frames.append(df_current_week)
    messages.append(msg_current)

    df_next_week, msg_next = process_week_with_gemini(
        START_DATE_NEXT, END_DATE_NEXT, WORKSHEET_NAME_NEXT, COLUMNS_NEXT, pdf_path
    )
    df_next_week = standardize_df(df_next_week, COLUMNS_NEXT)
    all_data_frames.append(df_next_week)
    messages.append(msg_next)

    df_all_new = pd.concat([d for d in all_data_frames if not d.empty], ignore_index=True)

    preview_df = pd.concat([df_current_week[COMMON_COLS], df_next_week[COMMON_COLS]], ignore_index=True)
    if preview_df.empty:
        final_message = "❌ 處理失敗，未能從 PDF 中提取任何課表資訊。"
    else:
        final_message = "\n".join(messages)

    return final_message, preview_df, df_all_new, df_current_week, df_next_week


# 查詢指定日期課程
def query_schedule(selected_date_label, df_current_state, df_next_state):

    selected_date_str = DATE_OPTIONS.get(selected_date_label, today.strftime("%Y-%m-%d"))

    if gc is None:
        return pd.DataFrame(columns=["課程名稱", "時段", "作業/攜帶品", "複習/預習章節", "備註"]), "❌ 錯誤：Google Sheets 服務未授權。", pd.DataFrame(), None

    try:
        target_date = dt_class.strptime(selected_date_str, "%Y-%m-%d").date()
    except ValueError:
        return pd.DataFrame(columns=["課程名稱", "時段", "作業/攜帶品", "複習/預習章節", "備註"]), "❌ 日期格式錯誤。", pd.DataFrame(), None

    is_current_week = (START_DATE_CURRENT <= target_date <= END_DATE_CURRENT)
    is_next_week = (START_DATE_NEXT <= target_date <= END_DATE_NEXT)

    df_current = standardize_df(load_sheet_to_df(WORKSHEET_NAME_CURRENT), COLUMNS_CURRENT)
    df_next = standardize_df(load_sheet_to_df(WORKSHEET_NAME_NEXT), COLUMNS_NEXT)

    target_df = pd.DataFrame()
    week_type = None

    if is_current_week and not df_current.empty:
        target_df = df_current[df_current["日期"] == target_date].copy().sort_values(["時間(起)"])
        week_type = WORKSHEET_NAME_CURRENT
    elif is_next_week and not df_next.empty:
        target_df = df_next[df_next["日期"] == target_date].copy().sort_values(["時間(起)"])
        week_type = WORKSHEET_NAME_NEXT

    if target_df.empty:
        return pd.DataFrame(columns=["課程名稱", "時段", "作業/攜帶品", "複習/預習章節", "備註"]), \
               f"🔍 {selected_date_str} 無課。", \
               pd.DataFrame(), None

    if week_type == WORKSHEET_NAME_CURRENT:
        col1_name, col2_name = "作業", "複習章節"
        input_col1_name, input_col2_name = "作業/攜帶品", "複習/預習章節)"
    else:
        col1_name, col2_name = "應帶用品", "預習章節"
        input_col1_name, input_col2_name = "作業/攜帶品", "複習/預習章節"

    display_df = pd.DataFrame()
    display_df["課程名稱"] = target_df["課程名稱"]
    display_df["時段"] = target_df["時間(起)"] + "-" + target_df["時間(迄)"]
    display_df["作業/攜帶品"] = target_df.get(col1_name, '')
    display_df["複習/預習章節"] = target_df.get(col2_name, '')
    display_df["備註"] = target_df.get("備註", '')

    df_input_to_save = target_df.copy()

    return display_df, f"✅ 成功查詢 {selected_date_str} ({week_type}) 課程。", df_input_to_save, week_type

# 回寫備註到 Sheets
def write_back_schedule(edited_df, df_input_to_save, week_type):
    default_df_return = (pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

    if df_input_to_save.empty or not week_type:
        return "❌ 錯誤：找不到當日課程或備註資訊。請先查詢課程並編輯表格。", *default_df_return

    TARGET_SHEET_NAME = week_type

    try:
        wk = gc.open_by_url(SHEET_URL)
        wk_sheet = wk.worksheet(TARGET_SHEET_NAME)
        all_values = wk_sheet.get_all_values()
        header = all_values[0]
        data_df = pd.DataFrame(all_values[1:], columns=header)
    except Exception as e:
        return f"❌ 讀取 Google Sheets 發生錯誤：{e}", *default_df_return

    try:
        if TARGET_SHEET_NAME == WORKSHEET_NAME_CURRENT:
            col1_name, col2_name = "作業", "複習章節"
        else:
            col1_name, col2_name = "應帶用品", "預習章節"

        col1_index = header.index(col1_name) + 1
        col2_index = header.index(col2_name) + 1
        col_備註_index = header.index("備註") + 1
    except ValueError:
        return f"### ❌ 錯誤：分頁 '{TARGET_SHEET_NAME}' 缺少必要的欄位。", *default_df_return

    updates = []

    if len(edited_df) != len(df_input_to_save):
        return "❌ 錯誤：編輯的行數與原始查詢結果不匹配，無法回寫。", *default_df_return

    edited_df_dict = edited_df.set_index('課程名稱').to_dict('index')

    for _, input_row in df_input_to_save.iterrows():
        match_course = input_row['課程名稱']

        edited_row = edited_df[edited_df['課程名稱'] == match_course].iloc[0]

        new_col1_value = str(edited_row["作業/攜帶品"]).strip()
        new_col2_value = str(edited_row["複習/預習章節"]).strip()
        new_備註 = str(edited_row['備註']).strip()

        match_date = str(input_row['日期'])

        for row_index, sheet_row in enumerate(data_df.values.tolist()):
            raw_sheet_date = sheet_row[header.index("日期")]
            sheet_course = sheet_row[header.index("課程名稱")]

            normalized_sheet_date = raw_sheet_date.replace("/", "-")

            if normalized_sheet_date == match_date and sheet_course == match_course:
                actual_row_num = row_index + 2

                if new_col1_value:
                    updates.append({'range': gspread.utils.rowcol_to_a1(actual_row_num, col1_index), 'values': [[new_col1_value]]})
                if new_col2_value:
                    updates.append({'range': gspread.utils.rowcol_to_a1(actual_row_num, col2_index), 'values': [[new_col2_value]]})
                if new_備註:
                    updates.append({'range': gspread.utils.rowcol_to_a1(actual_row_num, col_備註_index), 'values': [[new_備註]]})

                break

    if updates:
        wk_sheet.batch_update(updates)
        msg = f"✅ 成功更新 {len(updates)} 個單元格到 **'{TARGET_SHEET_NAME}'** 分頁中。"
    else:
        msg = "ℹ️ 沒有手動輸入的備註需要更新到 Sheets 中。"

    df_all_new, df_current_new, df_next_new = initial_data_load()

    return msg, df_all_new, df_current_new, df_next_new

In [140]:
def make_one_line_reminder(rows, is_current_week):
    if rows.empty: return ""

    r = rows.iloc[0]
    date_str = str(r.get("日期", "")).strip().replace("/", "-")
    weekday = str(r.get("星期", "")).strip()
    course_name = str(r.get("課程名稱", "")).strip()

    if is_current_week:
        item_col = "作業"
        read_col = "複習章節"
        item_label = "作業"
        read_label = "複習章節"
    else:
        item_col = "應帶用品"
        read_col = "預習章節"
        item_label = "攜帶品"
        read_label = "預習章節"

    item_value = str(r.get(item_col, '')).strip()
    read_value = str(r.get(read_col, '')).strip()
    memo_value = str(r.get("備註", '')).strip()

    item_output = f"{item_label}：{item_value if item_value else '無'}"
    read_output = f"{read_label}：{read_value if read_value else '無'}"
    memo_output = f"備註：{memo_value if memo_value else '無'}"

    return f"{date_str} {weekday} {course_name} {item_output} {read_output} {memo_output}"

# 生成提醒與 AI 資源
def generate_reminder_and_ai_resources(selected_date_label, df_current, df_next):
    if model is None:
        return "❌ 錯誤：Gemini API 未配置，無法生成 AI 內容。", "❌ 錯誤：Gemini API 未配置，無法生成 AI 內容。"

    selected_date_str = DATE_OPTIONS.get(selected_date_label, dt_class.now(TW_TZ).date().strftime("%Y-%m-%d"))

    try:
        target_date = dt_class.strptime(selected_date_str, "%Y-%m-%d").date()
    except ValueError:
        return "❌ 日期格式錯誤。", "❌ 日期格式錯誤。"

    is_current_week = (START_DATE_CURRENT <= target_date <= END_DATE_CURRENT)
    is_next_week = (START_DATE_NEXT <= target_date <= END_DATE_NEXT)

    if is_current_week and not df_current.empty:
        target_df = df_current[df_current["日期"] == target_date].copy().sort_values(["時間(起)"])
        week_type = WORKSHEET_NAME_CURRENT
    elif is_next_week and not df_next.empty:
        target_df = df_next[df_next["日期"] == target_date].copy().sort_values(["時間(起)"])
        week_type = WORKSHEET_NAME_NEXT
    else:
        return f"🔍 {selected_date_str} 不在目前課表範圍內或課表為空。", "ℹ️ 無法生成資源推薦。"

    if target_df.empty:
        return f"🔍 {selected_date_str} 無課，不需準備。", "ℹ️ 今日無課程，無需推薦資源。"

    reminders = []
    for _, row in target_df.iterrows():
        single_course_df = pd.DataFrame([row])
        one_line = make_one_line_reminder(single_course_df, is_current_week)
        if one_line:
            reminders.append(one_line)

    reminder_output_text = "今日無課程或數據錯誤。"
    if reminders:
        reminder_output_text = f"#### 📝 最終行前提醒（{week_type}）\n" + "\n".join([f"- {r}" for r in reminders])

    ai_recommendation_text = "### 💡 AI 課程資源推薦\n"
    course_list = target_df['課程名稱'].unique().tolist()

    for course in course_list:
        try:
            prompt = f"""
            請使用 **google:search** 工具查找並為課程 "{course}" 提供一個具體的學習資源推薦。

            推薦的內容必須包含以下兩點：
            1. 一個相關的 **YouTube 影片連結** (需包含影片標題)。
            2. 一個相關的 **延伸閱讀文章/網站連結** (需包含文章/網站標題)。

            請以以下 Markdown 格式輸出，不要有任何其他文字說明：

            #### 📚 【{course}】 推薦資源

            **YouTube 推薦**
            - [影片標題](影片連結)

            **延伸閱讀**
            - [文章/網站標題](文章/網站連結)
            """
            response = model.generate_content(prompt)

            ai_recommendation_text += response.text.strip() + "\n\n---\n"

        except Exception as e:
            ai_recommendation_text += f"#### 📚 【{course}】 推薦資源\n查找失敗: {e}\n\n---\n"

    return reminder_output_text, ai_recommendation_text

In [141]:
initial_df_all, initial_df_current, initial_df_next = initial_data_load()
initial_df_input_to_save = pd.DataFrame()
initial_week_type = None

with gr.Blocks() as demo:
    gr.Markdown("## 🗓️ 課表管理與行前提醒系統")

    df_all_state = gr.State(initial_df_all)
    df_current_state = gr.State(initial_df_current)
    df_next_state = gr.State(initial_df_next)
    df_input_to_save_state = gr.State(initial_df_input_to_save)
    week_type_state = gr.State(initial_week_type)

    with gr.Tab("🔄 課表更新與 PDF 上傳"):
        gr.Markdown("### 上傳新 PDF 課表並更新 Sheets 分頁")

        with gr.Row():
            pdf_uploader = gr.File(
                label="上傳您的新 PDF 課表檔案",
                file_types=[".pdf"],
                scale=2
            )

        upload_button = gr.Button("🚀 提取課表並回寫 Sheets (自動更新本週與下週)", variant="primary")

        upload_message = gr.Textbox(label="處理結果將顯示於此", lines=3, interactive=False)

        gr.Markdown("#### 成功提取的數據預覽")
        preview_df = gr.Dataframe(
            value=pd.DataFrame(columns=COMMON_COLS),
            label="提取的數據預覽",
            interactive=False
        )

        upload_button.click(
            handle_pdf_upload,
            inputs=[pdf_uploader],
            outputs=[upload_message, preview_df, df_all_state, df_current_state, df_next_state]
        )


    with gr.Tab("📝 課程備註與回寫"):
        gr.Markdown("### 查詢課程並輸入備註")

        with gr.Row():
            date_selector = gr.Dropdown(
                label="選擇您要查詢的日期",
                choices=list(DATE_OPTIONS.keys()),
                value=list(DATE_OPTIONS.keys())[0],
                interactive=True,
                scale=1
            )

        output_df = gr.DataFrame(
            value=pd.DataFrame(columns=["課程名稱", "時段", "作業/攜帶品", "複習/預習章節", "備註"]),
            headers=["課程名稱", "時段", "作業/攜帶品", "複習/預習章節", "備註"],
            label="課程列表與輸入 (請直接在表格中編輯)",
            interactive=True,
        )

        with gr.Row():
            query_button = gr.Button("🔍 查詢當日課程 (更新表格)", scale=1)
            write_back_button = gr.Button("💾 將備註回寫到 Google Sheets", variant="primary", scale=1)

        write_back_output = gr.Textbox(label="回寫 Sheets 結果", lines=3, interactive=False)

        query_button.click(
            query_schedule,
            inputs=[date_selector, df_current_state, df_next_state],
            outputs=[output_df, write_back_output, df_input_to_save_state, week_type_state]
        )

        write_back_button.click(
            write_back_schedule,
            inputs=[output_df, df_input_to_save_state, week_type_state],
            outputs=[write_back_output]
        )


    with gr.Tab("💡 AI 課程資源推薦"):
        gr.Markdown("### 生成行前提醒與 AI 推薦資源")

        with gr.Row():
            date_selector_ai = gr.Dropdown(
                label="選擇日期 (與備註分頁同步)",
                choices=list(DATE_OPTIONS.keys()),
                value=list(DATE_OPTIONS.keys())[0],
                interactive=True,
                scale=1
            )
            ai_query_button = gr.Button("🧠 生成提醒與 AI 資源推薦", variant="secondary", scale=1)

        reminder_output = gr.Markdown("最終行前提醒將顯示於此", visible=True)
        ai_output = gr.Markdown("AI 課程資源推薦將顯示於此", visible=True)

        ai_query_button.click(
            generate_reminder_and_ai_resources,
            inputs=[date_selector_ai, df_current_state, df_next_state],
            outputs=[reminder_output, ai_output]
        )

In [142]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d28576031735a58f48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d28576031735a58f48.gradio.live
